# Obter resultados de cnpjs em massa

In [2]:
import requests
import json
import os
import pandas as pd
import openpyxl	

planilha = openpyxl.load_workbook('lista_cnpj.xlsx')
cnpjs = planilha.active

df_resultados = pd.DataFrame(columns=['Nome','Nome Fantasia','cnpj',
                                      'Tipo','Porte','Natureza Jurídica','Atividade Principal', 
                                      'Atividades Secundarias', 'Socios','Capital Social','Endereço',
                                     'Email','Telefone'])

l_nome = []
l_nome_fantasia = []
l_cnpj = []
l_tipo = []
l_porte = []
l_natureza_juridica = []
l_atividade_principal = []
l_atividades_secundarias = []
l_socios = []
l_capital_social = []
l_endereco = []
l_email = []
l_telefone = []

c= 2
for i in cnpjs:
    valor = cnpjs["A"+str(c)].value
    if valor != None:
        url = requests.get(f'https://www.receitaws.com.br/v1/cnpj/{valor}').content
        rsult = json.loads(url.decode('utf-8'))
        
        l_nome.append(rsult['nome'])
        l_nome_fantasia.append(rsult['fantasia'])
        l_cnpj.append(rsult['cnpj'])
        l_tipo.append(rsult['tipo'])
        l_porte.append(rsult['porte'])
        l_natureza_juridica.append(rsult['natureza_juridica'])
        l_atividade_principal.append(rsult['atividade_principal'][0]['text'])

        cont = 0
        tam = len(rsult['atividades_secundarias'])
        atividades = ''
        while cont < tam:
            atividades+= rsult['atividades_secundarias'][cont]['text'] + ' - '
            cont+=1
        l_atividades_secundarias.append(atividades)

        con = 0
        ta = len(rsult['qsa']) 
        socios = ''
        while con < ta:
            socios+= f"{rsult['qsa'][c]['nome']} - {rsult['qsa'][c]['qual']} - "
            con+=1
        l_socios.append(socios)

        l_capital_social.append(rsult['capital_social'])
        l_endereco.append(f"{rsult['logradouro']} - {rsult['numero']} - {rsult['complemento']} - {rsult['bairro']} - {rsult['municipio']} - {rsult['uf']}")
        l_email.append(rsult['email'])
        l_telefone.append(rsult['telefone'])
    
    c+=1

df_resultados['Nome'] = l_nome
df_resultados['Nome Fantasia'] = l_nome_fantasia
df_resultados['cnpj'] = l_cnpj
df_resultados['Tipo'] = l_tipo
df_resultados['Porte'] = l_porte
df_resultados['Natureza Jurídica'] = l_natureza_juridica
df_resultados['Atividade Principal'] = l_atividade_principal
df_resultados['Atividades Secundarias'] = l_atividades_secundarias
df_resultados['Socios'] = l_socios
df_resultados['Capital Social'] = l_capital_social
df_resultados['Endereço'] = l_endereco
df_resultados['Email'] = l_email
df_resultados['Telefone'] = l_telefone

df_resultados.to_excel('resultado_cnpjs.xlsx', index=False)
os.startfile('resultado_cnpjs.xlsx')